# Langchain Tool Calling Agent

https://python.langchain.com/v0.1/docs/modules/agents/
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/

**Note**
This re-implements the "Single step agent" discussed earlier. In this implementation Langchain classes are getting utilized.

In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool

## Setup LLM for testing


In [2]:
from dotenv import load_dotenv
import sys
import json

from langchain.prompts import PromptTemplate

# Load the file that contains the API keys - OPENAI_API_KEY
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

# setting path
sys.path.append('../')

from utils.create_chat_llm import create_gpt_chat_llm, create_cohere_chat_llm, create_anthropic_chat_llm

# Try with GPT
llm = create_cohere_chat_llm()

## 1. Setup the prompt

To use the *Agent* class you need to provide a *prompt*. The prompt MUST have an input variable with the name {agent_scratchpad}

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{question}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


## 2. Setup tools

In [10]:
@tool
def company_stock_price(stock_symbol: str) -> float:
    """
    Retrieve the current stock price for a given company symbol.

    This function returns the stock price for specific company symbols. 
    Currently, it supports the following symbols:
    - 'AAPL' for Apple Inc.
    - 'MSFT' for Microsoft Corporation
    - 'AMZN' for Amazon.com, Inc.

    Args:
        stock_symbol (str): The stock symbol of the company.

    Returns:
        dict: A dictionary containing the stock price for the given company symbol.
              The dictionary has the following structure:
              {
                  "price": float | str
              }
              - "price" (float): The current stock price if the symbol is recognized.
              - "price" (str): "unknown" if the symbol is not recognized.

    Example:
        >>> company_stock_price('AAPL')
        {'price': 192.32}

        >>> company_stock_price('GOOGL')
        {'price': 'unknown'}
    """
    if stock_symbol.upper() == 'AAPL':
        return {"price": 192.32}
    elif stock_symbol.upper() == 'MSFT':
        return {"price": 415.60}
    elif stock_symbol.upper() == 'AMZN':
        return {"price": 183.60}
    else:
        return {"price": "unknown"}


In [11]:
@tool
def city_weather(city: str) -> dict:
    """
    Retrieve the current weather information for a given city.

    This function returns the weather information for specific cities. 
    Currently, it supports the following cities:
    - 'New York' with the forecast and temperature.
    - 'Paris' with the forecast and temperature.
    - 'London' with the forecast and temperature.

    Args:
        city (str): The name of the city.

    Returns:
        dict: A dictionary containing the weather information for the given city.
              The dictionary has the following structure:
              {
                  "temperature": int | str,
                  "forecast": str
              }
              - "temperature" (int): The current temperature in Fahrenheit if the city is recognized.
              - "forecast" (str): The weather forecast if the city is recognized.
              - "temperature" (str): "unknown" if the city is not recognized.

    Example:
        >>> city_weather('New York')
        {'temperature': 68, 'forecast': 'rain'}

        >>> city_weather('Tokyo')
        {'temperature': 'unknown'}
    """
    if city.lower() == "new york":
        return {"temperature": 68, "forecast": "rain"}
    elif city.lower() == "paris":
        return {"temperature": 73, "forecast": "sunny"}
    elif city.lower() == "london":
        return {"temperature": 82, "forecast": "cloudy"}
    else:
        return {"temperature": "unknown"}


## 3. Create the agent

In [12]:
tools = [company_stock_price, city_weather]

agent = create_tool_calling_agent(llm, tools, prompt)

## 4. Create the AgentExecutor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 5. Test

### 1. Simple test that uses 1 tool

In [14]:
question = "Which of these cities is hotter, Paris or London"
# question = "I am visting paris, should i carry an umbrella?"

response = agent_executor.invoke({"question": question})

print("Final response::",response)



> Entering new AgentExecutor chain...

Invoking: `city_weather` with `{'city': 'Paris'}`
responded: I will use the 'city_weather' tool to find out the temperature in Paris and London.

{'temperature': 73, 'forecast': 'sunny'}
Invoking: `city_weather` with `{'city': 'London'}`
responded: I will use the 'city_weather' tool to find out the temperature in Paris and London.

{'temperature': 82, 'forecast': 'cloudy'}London is hotter than Paris, with a temperature of 82 compared to Paris's 73.

> Finished chain.
Final response:: {'question': 'Which of these cities is hotter, Paris or London', 'output': "London is hotter than Paris, with a temperature of 82 compared to Paris's 73."}


### 2. Test for a scenario when no appropriate tool is available

In [15]:
question = "I have only $200, which stock can I invest in?"

response = agent_executor.invoke({"question": question})

print("Final response:",response)



> Entering new AgentExecutor chain...

Invoking: `company_stock_price` with `{'stock_symbol': 'AAPL'}`
responded: I will use the company_stock_price tool to find out the current stock price for Apple, Microsoft and Amazon. I will then write an answer based on this information.

{'price': 192.32}
Invoking: `company_stock_price` with `{'stock_symbol': 'MSFT'}`
responded: I will use the company_stock_price tool to find out the current stock price for Apple, Microsoft and Amazon. I will then write an answer based on this information.

{'price': 415.6}
Invoking: `company_stock_price` with `{'stock_symbol': 'AMZN'}`
responded: I will use the company_stock_price tool to find out the current stock price for Apple, Microsoft and Amazon. I will then write an answer based on this information.

{'price': 183.6}You could buy one share of Apple Inc. stock, which is currently trading at $192.32. You could also buy one share of Amazon stock, which is currently trading at $183.6. However, you could n

### 3. A complex scenario requiring use of more than one tool

In [16]:
question = """ 
I am interested in investing in one of these stocks [AAPL, MSFT, AMZN].
I will make my decision based on the weather forecast in the city I am in and the price of the stock.
If the weather is sunny then I will chose a stock that has the lowest price.
If the weather is raining then I will chose a stock that has the highest price.
If the wwather is cloudy then I will not buy any stock.

I am in new york, which stock should I invest in?

"""

response = agent_executor.invoke({"question": question})

print("Final response:",response)



> Entering new AgentExecutor chain...

Invoking: `city_weather` with `{'city': 'New York'}`
responded: I will find out the weather in New York and the stock prices of the three companies. I will then be able to answer the question.

{'temperature': 68, 'forecast': 'rain'}
Invoking: `company_stock_price` with `{'stock_symbol': 'AAPL'}`
responded: I will find out the weather in New York and the stock prices of the three companies. I will then be able to answer the question.

{'price': 192.32}
Invoking: `company_stock_price` with `{'stock_symbol': 'MSFT'}`
responded: I will find out the weather in New York and the stock prices of the three companies. I will then be able to answer the question.

{'price': 415.6}
Invoking: `company_stock_price` with `{'stock_symbol': 'AMZN'}`
responded: I will find out the weather in New York and the stock prices of the three companies. I will then be able to answer the question.

{'price': 183.6}It is currently raining in New York, so you should invest i